<a href="https://colab.research.google.com/github/bmnds/uea-data-science-04-data-processing/blob/main/%5BPreProcessing02%5D_Correlacao_entre_variaveis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [51]:
# Importando bibliotecas necessárias
import pandas as pd
import plotly.graph_objects as go
import plotly.offline as py
from scipy import stats

In [2]:
# Baixando os datasets
%%time
!curl -o datasets_aula2.tar.gz -s -OL 'https://drive.google.com/uc?export=download&id=1wu8uSaqMQ4kdJRoXUew8KoBHophG5lBk'
!tar -zxvf datasets_aula2.tar.gz

datasets_aula2/
datasets_aula2/countries.csv
datasets_aula2/Customers.csv
datasets_aula2/flags.csv
datasets_aula2/kc_house_data.csv
datasets_aula2/Orders.csv
CPU times: user 13.2 ms, sys: 7.83 ms, total: 21 ms
Wall time: 2.24 s


In [16]:
# Lendo as ordens
df = pd.read_csv('datasets_aula2/Orders.csv', sep=';', encoding='ISO-8859-1')
df.head()

,OrderID,CustomerID,EmployeeID,OrderDate,RequiredDate,ShippedDate,ShipVia,Freight,ShipName,ShipAddress,ShipCity,ShipRegion,ShipPostalCode,ShipCountry
0,10248,VINET,5,04/08/1994,01/09/1994,16/08/1994,3,"32,38",Vins et alcools Chevalier,59 rue de l'Abbaye,Reims,NaN,51100,France
1,10249,TOMSP,6,05/08/1994,16/09/1994,10/08/1994,1,"11,61",Toms Spezialitäten,Luisenstr. 48,Münster,NaN,44087,Germany
2,10250,HANAR,4,08/08/1994,05/09/1994,12/08/1994,2,"65,83",Hanari Carnes,"Rua do Paço, 67",Rio de Janeiro,RJ,05454-876,Brazil
3,10251,VICTE,3,08/08/1994,05/09/1994,15/08/1994,1,"41,34",Victuailles en stock,"2, rue du Commerce",Lyon,NaN,69004,France
4,10252,SUPRD,4,09/08/1994,06/09/1994,11/08/1994,2,"51,30",Suprêmes délices,"Boulevard Tirou, 255",Charleroi,NaN,B-6000,Belgium


In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 830 entries, 0 to 829
Data columns (total 14 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   OrderID         830 non-null    int64 
 1   CustomerID      830 non-null    object
 2   EmployeeID      830 non-null    int64 
 3   OrderDate       830 non-null    object
 4   RequiredDate    830 non-null    object
 5   ShippedDate     809 non-null    object
 6   ShipVia         830 non-null    int64 
 7   Freight         830 non-null    object
 8   ShipName        830 non-null    object
 9   ShipAddress     830 non-null    object
 10  ShipCity        830 non-null    object
 11  ShipRegion      323 non-null    object
 12  ShipPostalCode  811 non-null    object
 13  ShipCountry     830 non-null    object
dtypes: int64(3), object(11)
memory usage: 90.9+ KB


In [42]:
# Função para análise de correlação entre duas colunas
def analisar_correlacao(df, col1, col2):
  print(f'Analisando a correlação entre {col1} e {col2}:')
  cross = pd.crosstab(df[col1], df[col2])
  chi2, p, gl, f = stats.chi2_contingency(cross)
  if (p < 0.05):
    print(f'\t{col1} e {col2} São altamente Correlacionados (p={p})')
    print(f'\t{col2} é considerada Redundante e pode ser removida do dataframe')
  else:
    print(f'\t{col1} e {col2} Não possuem Correlação (p={p})')
    print(f'\t{col2} deve ser mantida no dataframe')

In [43]:
# Analisando a correlação entre ShipCountry e ShipCity
analisar_correlacao(df, 'ShipCountry', 'ShipCity')
# Analisando a correlação entre Employee e Ship Country
analisar_correlacao(df, 'ShipCountry', 'ShipVia')
# Analisando a correlação entre Employee e Customer?
analisar_correlacao(df, 'ShipName', 'CustomerID')
# Analisando a correlação entre Ship Name e Ship Country ?
analisar_correlacao(df, 'ShipName', 'ShipCountry')

Analisando a correlação entre ShipCountry e ShipCity:
	ShipCountry e ShipCity São altamente Correlacionados (p=0.0)
	ShipCity é considerada Redundante e pode ser removida do dataframe
Analisando a correlação entre ShipCountry e ShipVia:
	ShipCountry e ShipVia Não possuem Correlação (p=0.0984146997013663)
	ShipVia deve ser mantida no dataframe
Analisando a correlação entre ShipName e CustomerID:
	ShipName e CustomerID São altamente Correlacionados (p=0.0)
	CustomerID é considerada Redundante e pode ser removida do dataframe
Analisando a correlação entre ShipName e ShipCountry:
	ShipName e ShipCountry São altamente Correlacionados (p=0.0)
	ShipCountry é considerada Redundante e pode ser removida do dataframe


In [52]:
# Lendo dataset de casas
df = pd.read_csv('datasets_aula2/kc_house_data.csv')
df.sample(n=10)

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
13465,1822069052,20140709T000000,450000.0,5,2.50,2850,209523,1.0,0,0,4,7,1930,920,1925,1968,98058,47.3939,-122.089,2220,209523
11890,1523089012,20141120T000000,365000.0,4,1.00,1520,80150,1.0,0,0,2,5,1520,0,1948,0,98045,47.4742,-121.769,1740,84506
18886,8127700410,20141015T000000,511200.0,4,1.75,1480,7875,1.0,0,0,3,7,740,740,1927,0,98199,47.6430,-122.397,1680,5851
19925,6056111370,20141124T000000,340000.0,2,1.75,1270,1916,2.0,0,0,3,8,1270,0,2012,0,98108,47.5648,-122.294,1140,1916
6170,2652501630,20150504T000000,626700.0,3,1.50,1410,3600,1.0,0,0,3,7,1410,0,1906,0,98109,47.6402,-122.357,1370,3600
18560,955000453,20150413T000000,574000.0,2,2.25,1100,1114,2.0,0,0,3,8,900,200,2009,0,98122,47.6199,-122.304,1230,1800
19475,3755100540,20140725T000000,431200.0,5,1.75,1360,10609,1.0,0,0,3,7,1060,300,1966,0,98034,47.7203,-122.229,1490,9935
9600,8122600165,20141015T000000,273000.0,3,1.00,1500,6250,1.0,0,0,3,6,890,610,1945,0,98126,47.5365,-122.368,1210,6250
11526,3629860160,20141028T000000,825000.0,3,2.50,3760,5260,2.0,0,0,3,9,3230,530,2002,0,98029,47.5489,-122.007,3080,5312
20437,993001914,20150106T000000,344000.0,3,2.25,1250,1033,3.0,0,0,3,8,1250,0,2007,0,98103,47.6907,-122.343,1250,1150


In [53]:
# Analisando Preço x Qtd de Quartos
data = go.Scatter(x = df['price'],
                   y = df['bedrooms'],
                   mode = 'markers')
py.iplot([data])

In [55]:
df[['price', 'bedrooms']].cov()

,price,bedrooms
price,1.347824e+11,105286.276362
bedrooms,1.052863e+05,0.865015


In [57]:
df[['price', 'bedrooms']].corr(method = 'pearson')

,price,bedrooms
price,1.00000,0.30835
bedrooms,0.30835,1.00000


In [56]:
df[['price', 'bedrooms']].corr(method = 'kendall')

,price,bedrooms
price,1.000000,0.265094
bedrooms,0.265094,1.000000


In [58]:
df[['price', 'bedrooms']].corr(method = 'spearman')

,price,bedrooms
price,1.000000,0.344652
bedrooms,0.344652,1.000000


In [68]:
def analisar_covariancia(df, col1, col2):
  print(f'Analisando a covariância entre {col1} e {col2}')
  data = go.Scatter(x = df[col1],
                   y = df[col2],
                   mode = 'markers')
  py.iplot([data])
  print('\n#Covariância')
  print(df[[col1, col2]].cov())
  print('\n#Kendall')
  print(df[[col1, col2]].corr(method = 'kendall'))
  print('\n#Pearson')
  print(df[[col1, col2]].corr(method = 'pearson'))
  print('\n#Spearman')
  print(df[[col1, col2]].corr(method = 'spearman'))

In [70]:
# Análise de Área do lote em 2015 x Metragem quadrada da casa, além do porão
analisar_covariancia(df, 'sqft_lot15', 'sqft_above')

Analisando a covariância entre sqft_lot15 e sqft_above



#Covariância
              sqft_lot15    sqft_above
sqft_lot15  7.455182e+08  4.387534e+06
sqft_above  4.387534e+06  6.857347e+05

#Kendall
            sqft_lot15  sqft_above
sqft_lot15    1.000000    0.172943
sqft_above    0.172943    1.000000

#Pearson
            sqft_lot15  sqft_above
sqft_lot15     1.00000     0.19405
sqft_above     0.19405     1.00000

#Spearman
            sqft_lot15  sqft_above
sqft_lot15    1.000000    0.254203
sqft_above    0.254203    1.000000


In [69]:
# Análise de Preço x Qtd de Banheiros
analisar_covariancia(df, 'price', 'bathrooms')

Analisando a covariância entre price e bathrooms



#Covariância
                  price      bathrooms
price      1.347824e+11  148481.495749
bathrooms  1.484815e+05       0.593151

#Kendall
              price  bathrooms
price      1.000000   0.369208
bathrooms  0.369208   1.000000

#Pearson
              price  bathrooms
price      1.000000   0.525138
bathrooms  0.525138   1.000000

#Spearman
             price  bathrooms
price      1.00000    0.49716
bathrooms  0.49716    1.00000
